***The Text-Summarizing Model*** 


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np

In [112]:
nlp = English()
#A simple pipeline 
#component to allow custom sentence boundary detection logic that doesn’t require the dependency parse.
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [113]:
text_corpus = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [114]:
doc = nlp(text_corpus.replace("\n", ""))
sentences = [sent.string.strip() for sent in doc.sents]

In [115]:
print("Senetence are: \n", sentences)

Senetence are: 
 ['New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.', 'A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.', 'Only 18 days after that marriage, she got hitched yet again.', 'Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.', 'In 2010, she married once more, this time in the Bronx.', 'In an application for a marriage license, she stated it was her "first and only" marriage.', 'Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the2010 marriage license application, according to court documents.', 'Prosecutors said the marriages were part of an immigration scam.', 'On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment

In [116]:
# Let's create an organizer which will store the sentence ordering to later reorganize the 
# scored sentences in their correct order
sentence_organizer = {k:v for v,k in enumerate(sentences)}

In [117]:
print("Our sentence organizer: \n", sentence_organizer)

Our sentence organizer: 
 {'New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.': 0, 'A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.': 1, 'Only 18 days after that marriage, she got hitched yet again.': 2, 'Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.': 3, 'In 2010, she married once more, this time in the Bronx.': 4, 'In an application for a marriage license, she stated it was her "first and only" marriage.': 5, 'Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the2010 marriage license application, according to court documents.': 6, 'Prosecutors said the marriages were part of an immigration scam.': 7, 'On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christophe

In [118]:
# Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                    strip_accents='unicode', 
                                    analyzer='word',
                                    token_pattern=r'\w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=1,smooth_idf=1,
                                    sublinear_tf=1,
                                    stop_words = 'english')

In [119]:
# Passing our sentences treating each as one document to TF-IDF vectorizer
tf_idf_vectorizer.fit(sentences)

TfidfVectorizer(min_df=2, ngram_range=(1, 3), smooth_idf=1,
                stop_words='english', strip_accents='unicode', sublinear_tf=1,
                token_pattern='\\w{1,}', use_idf=1)

In [120]:
# Transforming our sentences to TF-IDF vectors
sentence_vectors = tf_idf_vectorizer.transform(sentences)

In [121]:
# Getting sentence scores for each sentences
sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

# checking up if we are going in right direction
print(len(sentences) == len(sentence_scores))
print(len(sentences))

True
20


In [122]:
# Getting top-n sentences
def n_value(sentences_length):
  if sentences_length>=1 and sentences_length<12:
    return 3
  else:
    return 4
N = n_value(len(sentences))
top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]

In [123]:
# Let's now do the sentence ordering using our sentence_organizer
# Let's map the scored sentences with their indexes
mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
print("Our top_n_sentence with their index: \n")
for element in mapped_top_n_sentences:
    print(element)

# Ordering our top-n sentences in their original ordering
mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

# Our final summary
summary = " ".join(ordered_scored_sentences)

Our top_n_sentence with their index: 

('New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.', 0)
('A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.', 1)
('In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.All occurred either in Westchester County, Long Island, New Jersey or the Bronx.', 10)
('Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the2010 marriage license application, according to court documents.', 6)


In [125]:
print("Summary: \n", summary)

Summary: 
 New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband. Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the2010 marriage license application, according to court documents. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.All occurred either in Westchester County, Long Island, New Jersey or the Bronx.


In [126]:
print(len(summary))

633


In [127]:
print(len(text_corpus))

2298
